In [2]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import geopandas as gpd
import ee
import geemap

from tsmoothie.smoother import LowessSmoother
# ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [2]:
def smooth(serie):
    serieResult = pd.Series(serie.values)
    serieindex = pd.Series(serie.index)
    smoother = LowessSmoother(smooth_fraction=0.1, iterations=2)

    dataSmooth = smoother.smooth(serieResult.values)
    return pd.Series(dataSmooth.smooth_data[0],index=serieindex)
    
def hampel_filter_pandas(input_series, window_size, n_sigmas=1):
    k = 1.4826 # Gaussian distribution
    new_series = input_series.copy()
    MAD = lambda x: np.median(np.abs(x - np.median(x)))
    rolling_median = input_series.rolling(window=2*window_size, center=True).mean()
    rolling_mad = k * input_series.rolling(window=2*window_size, center=True).apply(MAD)
    diff = (input_series - rolling_median) * -1 # toma solo las diff negativas
    indices = list(np.argwhere(diff.values > (n_sigmas * rolling_mad.values)).flatten())
    new_series[indices] = rolling_median[indices]
    return new_series#, indices

def hampel_filter_pandas_positivas(input_series, window_size, n_sigmas=1):
    k = 1.4826 
    new_series = input_series.copy()
    MAD = lambda x: np.median(np.abs(x - np.median(x)))
    rolling_median = input_series.rolling(window=2*window_size, win_type = 'gaussian', center=True).mean(std=1)
    rolling_mad = k * input_series.rolling(window=2*window_size, center=True).apply(MAD)
    diff = (input_series - rolling_median)
    indices = list(np.argwhere(diff.values > (n_sigmas * rolling_mad.values)).flatten())
    new_series[indices] = rolling_median[indices]
    return new_series

def cloud (image): #https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
    image = ee.Image(image)
    clouds = ee.Image(ee.List(image.get('cloud_mask'))).select('probability')
    clean = clouds.lte(80)
    qa = image.select('QA60')
    scl = image.select('SCL').eq(4).Or(image.select('SCL').eq(5))
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    finalmask = mask.And(clean)
    return image.updateMask(finalmask)

imageCollection = ee.ImageCollection("COPERNICUS/S2")
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask') \
                                    .apply(**{'primary': imageCollection,'secondary': s2Clouds, \
                                    'condition': ee.Filter.equals(**{'leftField': 'system:index', 'rightField': 'system:index'})})



# Aggregation
def aggreg (image_collection, startdate : str ,enddate : str ,timeunit : str)-> ee.ImageCollection:
    """ 
    image_collection: ee.ImageCollection
    startdate: string
    enddate: string
    timeunit: string 
    """
    start = ee.Date(startdate)
    end = ee.Date(enddate)
    timedelta = end.difference(start, timeunit)
    aggregation = 1
    def step (count):
        return start.advance(count, timeunit)
    def filter (date):
        images_g = ee.ImageCollection(image_collection.filterDate(date,ee.Date(date).advance(aggregation,timeunit)))
        return ee.Image(images_g.mean()).set('system:time_start',ee.Date(date)).set('system:date',ee.Date(date).format('dd-MM-yyyy'))
    S2_list = ee.List.sequence(0, timedelta.int(),aggregation) \
        .map(step) \
        .map(filter)

    # Check missing values
    def check (image):
        image = ee.Image(ee.Algorithms.If(ee.Image(image).bandNames(), ee.Image(image)))
        return image
    return ee.ImageCollection(S2_list).map(check, True)








In [3]:
gdf = gpd.read_file('Frog_selsey_outwoods.geojson')
gdf.crs = "epsg:4326"
table_json = gdf.__geo_interface__
table = ee.FeatureCollection(table_json['features'])

In [4]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [5]:
Map.add_basemap('SATELLITE')
Map.addLayer(table, {}, 'Farm')
Map.centerObject(table, 10)



In [13]:
# Map.draw_features.clear()

In [ ]:

roi = ee.FeatureCollection(Map.draw_features)
selected_fields = table.filterBounds(roi)
Map.addLayer(selected_fields, {}, "Selected states")

In [ ]:
def data_extr (image):
    dia = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd')
    ndvi = image.normalizedDifference(['B8','B4']).rename('ndvi')
    data = ndvi.reduceRegions(selected_fields ,ee.Reducer.mean(),10).filterMetadata('mean','greater_than',0)
    field_name = data.aggregate_array('Field')
    farm_name = data.aggregate_array('Farm')
    crop_id = 'N/A' #data.aggregate_array('crop_id')
    ndvi_value = ee.List(data.aggregate_array('mean'))
    dia_list = ee.List.repeat(dia,ndvi_value.length())
    return ee.Feature(None).set('field',field_name,'farm',farm_name,'ndvi_mean',ndvi_value,'date',dia_list,'crop_id',crop_id)

images = s2SrWithCloudMask.filterBounds(selected_fields).map(cloud)
S2_list_clean = aggreg(images ,'2017-01-01','2022-01-01','month') #'day' or 'month'
image_list = ee.List(S2_list_clean.aggregate_array('system:time_start')).map(lambda date : ee.Date(date).format('yyyy-MM-dd')).getInfo()
list_dates = list(dict.fromkeys(image_list))
data = ee.ImageCollection(S2_list_clean).map(data_extr).getInfo()




In [ ]:
# display(list_dates)

#### obtain list fields
fields = selected_fields.getInfo()


list_fields=[]
for feat in fields['features']:
    field = feat['properties']['Field']
    list_fields.append(field)

list_farms=[]
for feat in fields['features']:
    crop_id = feat['properties']['Farm']
    list_farms.append(crop_id)

list_tuples = list(zip(set(list_fields),list_farms))

In [ ]:
x  = pd.DatetimeIndex(list_dates).sort_values()
df = pd.DataFrame(columns = set(list_fields), index = x).astype(float) 
for feat in data['features']:
    field_name = pd.Series(feat['properties']['field'])
    fechas = pd.Series(feat['properties']['date'],dtype='datetime64[ns]')
    ndvi_mean = feat['properties']['ndvi_mean']
    if fechas.size != 0 :
        df.loc[fechas,field_name] = ndvi_mean



df_int = df.interpolate( method='time', limit_direction ='both').dropna()

df_hampel = pd.DataFrame()
df_hampel_smooth = pd.DataFrame()
for column in df_int:
    df_hampel[column] = hampel_filter_pandas(df_int[column],2,0.5)
    df_hampel[column] = hampel_filter_pandas_positivas(df_hampel[column],2,2)
    df_hampel_smooth[column] = smooth(df_hampel[column])
    
df_hampel_smooth = df_hampel_smooth.interpolate( method='linear', limit_direction ='both').dropna()

In [ ]:
import plotly.express as px
df_plot = df_hampel_smooth
fied_names = df_plot.columns.get_level_values(0)
df_plot.columns = fied_names

fig = px.line(df_plot, x=df_plot.index, y=df_plot.columns , range_y=[0,1])
fig.show(figsize=(20,10))